In [15]:
#Read completion data
import json

def prep_completions(completion_file):
    completion_list = [] #output 2d list of prompt completions
    with open(completion_file, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, start=1):
            try:
                data = json.loads(line)
                passage_set = []
                completions = data['completions']
                for completion in completions:
                    passage =  json.loads(completion)['message']['content']
                    passage_set.append(passage)
                completion_list.append(passage_set)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {line_number}: {e}")
    return completion_list

medqa_completion_file = 'medqa_generated_responses.jsonl'
medqa_completion_file_b = 'medqa_generated_responses_b.jsonl'
civilqa_completion_file = 'civilqa_generated_responses.jsonl'
civilqa_completion_file_b = 'civilqa_generated_responses_b.jsonl'

chatgpt_completion_file = "chatgpt_generated_responses_b.jsonl"
chatgpt_med_completion_file = "chatgpt_meda_generated_responses.jsonl"
chatgpt_civil_completion_file = "chatgpt_civila_generated_responses.jsonl"

med_civil_completion_file = "medq_civila_generated_responses.jsonl"
civil_med_completion_file = "civilq_meda_generated_responses.jsonl"

logic_chatgpt_completion_file = "logicqa_generated_responses_b.jsonl"
logic_med_completion_file = "logicq_meda_generated_responses.jsonl"
logic_civil_completion_file = "logicq_civila_generated_responses.jsonl"




#privacyqa_completion_list = prep_completions(privacyqa_completion_file)
completion_list = prep_completions(logic_civil_completion_file)

print(f"Created list of {len(completion_list)} completion sets")


Created list of 400 completion sets


In [16]:
#Collect sentence level BERTScore
import torch
import time
import spacy
from selfcheckgpt.modeling_selfcheck import SelfCheckBERTScore
from bert_score import score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device ", device)
selfcheck_bertscore = SelfCheckBERTScore(rescale_with_baseline=True)
nlp = spacy.load("en_core_web_sm")

medqa_results_file='medqa_bertscore_results.txt'
medqa_results_file_b='medqa_bertscore_results_b.txt'
civilqa_results_file='civilqa_bertscore_results.txt'
civilqa_results_file_b='civilqa_bertscore_results_b.txt'

chatgpt_results = 'chatgpt_bertscore_results.txt'
chatgpt_med_results = "chatgpt_meda_bertscore_results.txt"
chatgpt_civil_results = "chatgpt_civila_bertscore_results.txt"

med_civil_results = "medq_civila_bertscore_results.txt"
civil_med_results = "civilq_meda_bertscore_results.txt"

logic_chatgpt_results = "logicqa_bertscore_results_b.txt"
logic_med_results = "logicq_meda_bertscore_results.txt"
logic_civil_results = "logicq_civila_bertscore_results.txt"



def doBERT(completion_list, output_path):
    results=[]
    count = 0
    with open(output_path, 'w') as outfile:
        for passages in completion_list:
            try:
                count += 1
                #print("try entry #", count, ": ", len(passages)," passages.")
                sentences = [sent.text.strip() for sent in nlp(passages[0]).sents] # spacy sentence tokenization
                sent_scores_bertscore = selfcheck_bertscore.predict(
                    sentences = sentences,             # list of sentences
                    sampled_passages = passages, # list of sampled passages
                )
                results.append(sent_scores_bertscore)
                line_array = ' '.join(map(str, sent_scores_bertscore))
                outfile.write(line_array+ '\n')
            except Exception as e:
                print(f"Error processing BERTScore: \n{e}")
                time.sleep(.05)  # Brief pause in case
    return results
   
score_results = doBERT(completion_list, logic_civil_results)




using device  cpu
SelfCheck-BERTScore initialized
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of range
Error processing BERTScore: 
list index out of ra

In [3]:
#Get Mean BERTScore
import numpy as np

def mean_score(filename):
    averages_list=[]
    with open(filename, 'r') as file:
        for line in file:
            numbers_str = line.strip().split(' ')
            numbers = np.array(numbers_str, dtype=float)
            if numbers.size > 0:
                line_average = np.mean(numbers) 
                averages_list.append(line_average)
        if averages_list:
            mean = np.mean(averages_list)
            std = np.std(averages_list, ddof=1)
        else:
            mean = 0
            std = 0
    return mean, std

chatgpt_results = 'chatgpt_bertscore_results.txt'
medqa_results_file='medqa_bertscore_results.txt'
medqa_results_file_b='medqa_bertscore_results_b.txt'
civilqa_results_file='civilqa_bertscore_results.txt'
civilqa_results_file_b='civilqa_bertscore_results_b.txt'

chatgpt_results = 'chatgpt_bertscore_results.txt'
chatgpt_med_results = "chatgpt_meda_bertscore_results.txt"
chatgpt_civil_results = "chatgpt_civila_bertscore_results.txt"

med_civil_results = "medq_civila_bertscore_results.txt"
civil_med_results = "civilq_meda_bertscore_results.txt"

logic_chatgpt_results = "logicqa_bertscore_results_b.txt"
logic_med_results = "logicq_meda_bertscore_results.txt"
logic_civil_results = "logicq_civila_bertscore_results.txt"

print("Mean baseline score: ",mean_score(chatgpt_results))
print("Mean medqa score: ",mean_score(medqa_results_file))
print("Mean medqa baseline score: ",mean_score(medqa_results_file_b))
print("Mean civilqa score: ",mean_score(civilqa_results_file))
print("Mean civilqa baselinescore: ",mean_score(civilqa_results_file_b))

print("Mean chatgpt meda  model score: ",mean_score(chatgpt_med_results))
print("Mean chatgpt civila model score: ",mean_score(chatgpt_civil_results))

print("Mean medq civila model score: ",mean_score(med_civil_results))
print("Mean civilq meda model score: ",mean_score(civil_med_results))

print("Mean logicqa model score: ",mean_score(logic_chatgpt_results))
print("Mean logicq meda model score: ",mean_score(logic_med_results))
print("Mean logicq civila model score: ",mean_score(logic_civil_results))



Mean baseline score:  (0.4365593959120497, 0.057618705016107725)
Mean medqa score:  (0.43447367395166525, 0.17054266197225984)
Mean medqa baseline score:  (0.44198961447014484, 0.059013949266016066)
Mean civilqa score:  (0.6815828280007615, 0.1409255503057828)
Mean civilqa baselinescore:  (0.48734517240001635, 0.050970526031263196)
Mean chatgpt meda  model score:  (0.5878173037358698, 0.07429819562498428)
Mean chatgpt civila model score:  (0.6147738892576885, 0.119530184588773)
Mean medq civila model score:  (0.6266185919050965, 0.14333385186974607)
Mean civilq meda model score:  (0.6831255163270078, 0.1396715709574812)
Mean logicqa model score:  (0.36407658372282875, 0.09565135502754737)
Mean logicq meda model score:  (0.2984200845736373, 0.20998290958891355)
Mean logicq civila model score:  (0.26596914317204606, 0.2525991848451925)
